# Collaboration and Competition

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 3.0.5 which is incompatible.


The environment is already saved in the Workspace and can be accessed at the file path provided below. 

In [2]:
from unityagents import UnityEnvironment
import numpy as np
from maddpg_agent import Agent
from collections import deque
import matplotlib.pyplot as plt
import random
import torch
%matplotlib inline

env = UnityEnvironment(file_name="/data/Tennis_Linux_NoVis/Tennis")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.         -6.65278625 -1.5        -0.          0.
  6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [5]:
for i in range(5):                                         # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: -0.004999999888241291
Total score (averaged over agents) this episode: -0.004999999888241291
Total score (averaged over agents) this episode: -0.004999999888241291
Total score (averaged over agents) this episode: -0.004999999888241291
Total score (averaged over agents) this episode: -0.004999999888241291


### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

In [6]:
PRINT_EVERY = 25
def maddpg_train(agent_0, agent_1, n_episodes=4000, max_t=1000):
    """Multi-Agent Deep Deterministic Policy Gradient (MADDPG)"""
    
    scores_window = deque(maxlen=100)
    scores_all = []
    best_score = -np.inf
    best_episode = 0
    already_solved = False    

    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        states = np.reshape(env_info.vector_observations, (1,48)) # get states and combine them
        agent_0.reset()
        agent_1.reset()
        scores = np.zeros(num_agents)
        while True:
            actions = get_actions(states, True) # Get concatenated actions of both the agents
            env_info = env.step(actions)[brain_name]
            next_states = np.reshape(env_info.vector_observations, (1, 48))
            rewards = env_info.rewards
            done = env_info.local_done
            agent_0.step(states, actions, rewards[0], next_states, done, 0)
            agent_1.step(states, actions, rewards[1], next_states, done, 1)
            scores += np.max(rewards) # Choose the higher reward as the score out of those two
            states = next_states                               
            if np.any(done):
                break

        ep_score = np.max(scores)
        scores_window.append(ep_score)
        scores_all.append(ep_score)

        # save best score                        
        if ep_score > best_score:
            best_score = ep_score
            best_episode = i_episode
        
        # print results
        if i_episode % PRINT_EVERY == 0:
            print('\rEpisode {}\tAverage Score: {:.3f}'.format(i_episode, np.mean(scores_window)))

        # determine if environment is solved and keep best performing models
        if np.mean(scores_window)>=0.5 and i_episode>=100:
            if not already_solved:
                print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.3f}'.format(i_episode-100, np.mean(scores_window)))
                already_solved = True
                # save weights
                torch.save(agent_0.actor_local.state_dict(), 'models/checkpoint_actor_0.pth')
                torch.save(agent_0.critic_local.state_dict(), 'models/checkpoint_critic_0.pth')
                torch.save(agent_1.actor_local.state_dict(), 'models/checkpoint_actor_1.pth')
                torch.save(agent_1.critic_local.state_dict(), 'models/checkpoint_critic_1.pth')
            elif ep_score >= best_score:
                print('Best episode so far with Average Score: {:.3f} and episode score: {:.3f} after {:d} episodes'.format(
                np.mean(scores_window), ep_score, i_episode))
                # save weights
                torch.save(agent_0.actor_local.state_dict(), 'models/checkpoint_actor_0.pth')
                torch.save(agent_0.critic_local.state_dict(), 'models/checkpoint_critic_0.pth')
                torch.save(agent_1.actor_local.state_dict(), 'models/checkpoint_actor_1.pth')
                torch.save(agent_1.critic_local.state_dict(), 'models/checkpoint_critic_1.pth')
            elif (i_episode-best_episode) >= 100:
                # stop training if model stops converging
                print('<-- Training stopped. Best score not matched or exceeded for 100 episodes')
                break
            else:
                continue
            
    return scores_all


def get_actions(states, add_noise):
    action_0 = agent_0.act(states, add_noise)
    action_1 = agent_1.act(states, add_noise)
    return np.concatenate((action_0, action_1), axis=0).flatten()

In [7]:
# initialize agents
agent_0 = Agent(state_size, action_size, seed=0, num_agents=1)
agent_1 = Agent(state_size, action_size, seed=0, num_agents=1)

Running on: cuda:0
Running on: cuda:0


In [8]:
scores = maddpg_train(agent_0, agent_1)

Episode 25	Average Score: 0.020
Episode 50	Average Score: 0.020
Episode 75	Average Score: 0.032
Episode 100	Average Score: 0.039
Episode 125	Average Score: 0.049
Episode 150	Average Score: 0.058
Episode 175	Average Score: 0.067
Episode 200	Average Score: 0.067
Episode 225	Average Score: 0.067
Episode 250	Average Score: 0.058
Episode 275	Average Score: 0.050
Episode 300	Average Score: 0.051
Episode 325	Average Score: 0.060
Episode 350	Average Score: 0.073
Episode 375	Average Score: 0.068
Episode 400	Average Score: 0.073
Episode 425	Average Score: 0.071
Episode 450	Average Score: 0.076
Episode 475	Average Score: 0.087
Episode 500	Average Score: 0.097
Episode 525	Average Score: 0.118
Episode 550	Average Score: 0.139
Episode 575	Average Score: 0.144
Episode 600	Average Score: 0.154
Episode 625	Average Score: 0.146
Episode 650	Average Score: 0.137
Episode 675	Average Score: 0.141
Episode 700	Average Score: 0.153
Episode 725	Average Score: 0.171
Episode 750	Average Score: 0.174
Episode 775	A

FileNotFoundError: [Errno 2] No such file or directory: 'models/checkpoint_actor_0.pth'

In [9]:
torch.save(agent_0.actor_local.state_dict(), 'models/checkpoint_actor_0.pth')
torch.save(agent_0.critic_local.state_dict(), 'models/checkpoint_critic_0.pth')
torch.save(agent_1.actor_local.state_dict(), 'models/checkpoint_actor_1.pth')
torch.save(agent_1.critic_local.state_dict(), 'models/checkpoint_critic_1.pth')